In [3]:
import sys
import glob
path='/mnt/e/Simulations/Facilitation/facilitation/newFac'
sys.path.append(path)
import singPartNew as sp
import detectInstantons as dI
import numpy as np
from scipy.interpolate import UnivariateSpline
from scipy.optimize import curve_fit
import pylab as pl
import pandas as pd
from tqdm.notebook import tqdm


### Specify the files and the parameters of the trajectory ###

In [4]:
fileName = glob.glob('/media/xn18583/Elements/allBC3/dataTrond/KA_21/glass/T0.48/forFascilitation/T0.48_N10002_NVT_KA21_step_1LJ_startFrame*20.xyz') 
print(len(fileName), ' files found.')
rho=1.4               # number density
N = 10002             # number of particles 
L  = (N/rho)**(1./3.) # box length (square box assumed)
ta = 200              # excitation fit length
a = 0.5               # threshold for excitation detection
numFrames = 1001    # number of frames



5  files found.


In [5]:
excitationIDs = pd.DataFrame(columns = [ 'fileID', 'particleID', 't_0', 'deltat','amplitude']) # results file
for xyzfile in fileName:
    fileID= str(int(xyzfile[xyzfile.find('startFrame')+10:xyzfile.find('.xyz')]))       # find the file ID (in my case frame from the run was started)
    instantonIDs,failCount = dI.detect(xyzfile,fileID,ta,a,N,numFrames,L)                  # find instantons in file
    print('In run ',fileID, 'the algorithm detected ',len(instantonIDs), 'excitations.')	
    print('The fit failed ',failCount[0],' times and deltat was too long ',failCount[1],' times!')
    excitationIDs = pd.concat([excitationIDs,instantonIDs])                             # add instantons detected in this file to results


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10002/10002 [00:27<00:00, 360.91it/s]


In run  220 the algorithm detected  22 excitations.
The fit failed  0  times and deltat was too long  0  times!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10002/10002 [01:14<00:00, 134.54it/s]


In run  20 the algorithm detected  73 excitations.
The fit failed  0  times and deltat was too long  0  times!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10002/10002 [00:46<00:00, 216.32it/s]


In run  120 the algorithm detected  40 excitations.
The fit failed  0  times and deltat was too long  2  times!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10002/10002 [00:34<00:00, 287.40it/s]


In run  320 the algorithm detected  30 excitations.
The fit failed  0  times and deltat was too long  0  times!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10002/10002 [00:35<00:00, 279.82it/s]

In run  420 the algorithm detected  32 excitations.
The fit failed  0  times and deltat was too long  0  times!


In [6]:
excitationIDs.to_csv('results/excitationNew_'+sys.argv[2]+'.csv', index=False)                # save results

FileNotFoundError: [Errno 2] No such file or directory: 'results/excitationNew_/home/xn18583/.local/share/jupyter/runtime/kernel-a45a591c-6eff-44e6-8f7e-d582c1a41ae2.json.csv'

In [ ]:
# inspect results
excitationIDs.head()

### Average number of excitations per frame ###

In [ ]:
excitationIDs.groupby('fileID').particleID.count().mean()

### Distribution of jump durations $\Delta t$ ###

Many more files are required at this temperature to obtain decent statistics

In [ ]:
pl.figure(figsize = (4,4))
bins = np.linspace(0, 100, 50)
centres = (bins[1:] + bins[:-1]) / 2
hist, cen = np.histogram(list(excitationIDs.deltat),bins = bins,density= True)
pl.semilogy(centres,hist,linewidth=4,color='black')
pl.semilogy(centres,hist,linewidth=3,color='blue')
pl.xlabel(r'$\Delta t$ [LJ]')
pl.ylabel(r'$p(\Delta t)$')